EVRİŞİMLİ SİNİR AĞLARI (CNN) İLE KATI ATIK TESPİTİ

1) İş Problemi (Business Problem)

2) Veriyi Anlamak (Data Understanding)

3) Veriyi Hazırlamak (Data Preparation)

4) Modelleme (Modeling)

5) Değerlendirme (Evaluation)

# 1) İş Problemi (Business Problem)

İş Problemi Nedir?

Katı atıkların, cam - kağıt - karton - plastik - metal - çöp kategorilerine otonom bir şekilde ayrılması gerekmektedir.

# 2) Veriyi Anlamak (Data Understanding)

Veri Seti Hikayesi

Bu proje kapsamında kullanacağımız veri seti TrashNet isimli veri setidir. Stanford Üniversitesi öğrencileri tarafından hazırlanmıştır.
Veri seti altı farklı sınıftan oluşmaktadır. Veri setinde Cam, Kağıt, Karton, Plastik, Metal ve Çöp olmak üzere toplamda 2527 adet görüntü bulunmaktadır.

Görüntülerin dağılımı:
- 501 cam
- 594 kağıt
- 403 karton
- 482 plastik
- 410 metal
- 137 çöp

Görüntüler beyaz bir panoya yerleştirilerek ve güneş ışığı veya oda aydınlatması kullanılarak çekilmiştir. Görüntüler, 512x384 piksel boyutlarında ve 3 (RGB) kanallıdır.

### 2.1) Gerekli Kütüphanelerin Import İşlemleri

In [1]:
!pip install imutils

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25859 sha256=4daa70721a76e65d0a51432f2e4763ac38bd5d78705c88d607548ecb8f30903d
  Stored in directory: c:\users\suheyl\appdata\local\pip\cache\wheels\f4\4b\8b\a9c23da464a09c6ad0a131c1752079bc85f9f1677c7b78e87d
Successfully built imutils



[notice] A new release of pip available: 22.3 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
!pip install opencv-python

     ---------------------------------------- 38.2/38.2 MB 2.7 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
# Veriyi okuma ve işleme adımında kullanılacak olan kütüphaneler

import cv2
import urllib
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import random, os, glob
from imutils import paths
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from urllib.request import urlopen

# Warningleri kapatmak için kullanılmaktadır.

import warnings
warnings.filterwarnings("ignore")

# Model değerlendirme için kullanılacak olan kütüphaneler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Conv2D, Flatten, MaxPooling2D, Dense, Dropout, SpatialDropout2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img



### 2.2) Veri Setinin Okunması

In [2]:
from google.colab import drive
drive.mount('/content/drive')

In [48]:
dir_path = "D:\\Süheyl\\Python_Docs\\Courses\\Turkcell_Geleceği_Yazanlar\\Data_Science_Course\\Python_for_DataScience-master\\Derin Öğrenme\\CNN\\_archive\\Garbage classification\\Garbage classification"


In [12]:
# Target size ve Label Değerlerinin Belirlenmesi

target_size = (224, 224)

waste_labels = {"cardboard": 0, 
                "glass": 1, 
                "metal" : 2, 
                "paper" : 3, 
                "plastic" : 4, 
                "trash" : 5}

In [46]:
def load_datasets(path):
    """
Görsellerin bulunduğu dizindeki görüntüyü okuyup etiketlerini oluşturur.
Parametreler:
path: Görsellerin bulunduğu dizini ifade eder.
Return:
x: Görüntülere ait matris bilgilerini tutar.
labels: Görüntünün ait olduğu sınıf bilgisini tutan liste.
    """


    x = []
    labels = []

    # Gönderdiğimiz pathdeki görüntüleri listeleyip sıralamaktadır.
    image_paths = sorted(list(paths.list_images(path)))

    for image_path in image_paths:
        # Belirtilen pathdeki görüntüler openCV kütüphanesi ile okunmaktadır.
        img = cv2.imread(image_path)
        
        # Okunan görüntüler başlangıçta belirlenen target_size'a göre yeniden ölçeklendirilir.
        img = cv2.resize(img, target_size)
        
        # Ölçeklendirilen görüntüler x listesine eklenir.
        x.append(img)
        
        # Her bir path "/" ifadesi ile ayrıldığında dönen listenin sonran ikinci elamanı label'ı temsil etmektedir.
        label = image_path.split(os.path.sep)[-2]
        
        # Yakalanan labelların sayısal değer karşılıklarının olduğu waste_labels sözlüğü içerisinden gönderilen key
        # değerine karşılık value değeri alınarak label oluşturulur.
        labels.append(waste_labels[label])

    # Veri seti random bir şekilde karıştırılır.
    x, labels = shuffle(x, labels, random_state=42)

    # Boyut ve Sınıf bilgisi raporlanmaktadır.
    print(f"X boyutu: {np.array(x).shape}")
    print(f"Label sınıf sayısı: {len(np.unique(labels))} Gözlem sayısı: {len(labels)}")

    return x, labels

In [3]:
x, labes = load_datasets(dir_path)